In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [95.2 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [983 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-20 02:53:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.17MB/s    in 0.8s    

2022-10-20 02:53:19 (1.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge2").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

### Filter Vote DataFrame using various criteria

In [28]:
# Filter Vote Dataframe to retrieve rows with >= 20 total votes
votes20_df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
votes20_df = votes20_df.filter(votes20_df["total_votes"] >=20)
votes20_df.show(20)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           19|         20|   N|                Y|
|          5|           19|         20|   N|                Y|
|          5|           30|         30|   N|                Y|
|          1|            8|         30|   N|                N|
|          5|           35|         37|   N|                Y|
|          5|           19|         22|   N|                Y|
|          5|           27|         28|   N|                Y|
|          4|           25|         26|   N|                Y|
|          1|           14|         20|   N|                Y|
|          5|           25|         27|   N|                Y|
|          5|           26|         28|   N|                Y|
|          5|           48|         49|   N|                N|
|          5|           19|         20|   N|           

In [37]:
# Filter New Dataframe created in previous step to retrieve rows where 'helpful_votes'/'total_votes >= 50%
helpfulvotes_df = votes20_df.filter(votes20_df["helpful_votes"]/votes20_df["total_votes"] >= 0.50)
helpfulvotes_df.show(20)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           19|         20|   N|                Y|
|          5|           19|         20|   N|                Y|
|          5|           30|         30|   N|                Y|
|          5|           35|         37|   N|                Y|
|          5|           19|         22|   N|                Y|
|          5|           27|         28|   N|                Y|
|          4|           25|         26|   N|                Y|
|          1|           14|         20|   N|                Y|
|          5|           25|         27|   N|                Y|
|          5|           26|         28|   N|                Y|
|          5|           48|         49|   N|                N|
|          5|           19|         20|   N|                Y|
|          5|           49|         52|   Y|           

In [30]:
# Filter New Dataframe created in previous step to retrieve rows containing a paid Vine program review
paidreview_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == 'Y' )
paidreview_df.show(20)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           49|         52|   Y|                N|
|          5|           34|         39|   Y|                N|
|          4|           37|         39|   Y|                N|
|          3|           18|         25|   Y|                N|
|          5|           44|         45|   Y|                N|
|          5|           26|         27|   Y|                N|
|          4|           43|         48|   Y|                N|
|          4|           20|         23|   Y|                N|
|          4|           27|         30|   Y|                N|
|          5|           26|         28|   Y|                N|
|          4|          349|        370|   Y|                N|
|          4|          107|        121|   Y|                N|
|          2|           22|         30|   Y|           

In [32]:
# Filter New Dataframe created 2 previous steps ago to retrieve rows containing a review not part of the Vine program 
nonpaidreview_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == 'N' )
nonpaidreview_df.show(20)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           19|         20|   N|                Y|
|          5|           19|         20|   N|                Y|
|          5|           30|         30|   N|                Y|
|          5|           35|         37|   N|                Y|
|          5|           19|         22|   N|                Y|
|          5|           27|         28|   N|                Y|
|          4|           25|         26|   N|                Y|
|          1|           14|         20|   N|                Y|
|          5|           25|         27|   N|                Y|
|          5|           26|         28|   N|                Y|
|          5|           48|         49|   N|                N|
|          5|           19|         20|   N|                Y|
|          5|           27|         28|   N|           

In [9]:
from pyspark.sql.functions import col, avg

### Number of reviews, number of 5-star reviews, percentage of 5-star reviews for each type

In [39]:
reviews = helpfulvotes_df.count()
fivestarreviews = helpfulvotes_df.filter(helpfulvotes_df["star_rating"] == '5').count()
vinereviews = helpfulvotes_df.filter(helpfulvotes_df["vine"] =='Y').count()
nonvinereviews = helpfulvotes_df.filter(helpfulvotes_df["vine"] =='N').count()
fivestarreviewsvine = helpfulvotes_df.filter((helpfulvotes_df["star_rating"] == '5') & (helpfulvotes_df["vine"] =='Y')).count()
fivestarreviewsnonvine = helpfulvotes_df.filter((helpfulvotes_df["star_rating"] == '5') & (helpfulvotes_df["vine"] =='N')).count()
percentvinefive = float(fivestarreviewsvine/vinereviews)
percentnonvinefive = float(fivestarreviewsnonvine/nonvinereviews)
print("The total count of watch reviews is "  + str(reviews))
print("The total count of 5 star watch reviews is "  + str(fivestarreviews))
print("The total count of Vine watch reviews is "  + str(vinereviews))
print("The total count of 5 star Vine watch reviews is "  + str(fivestarreviewsvine))
print("The total count of Non-Vine watch reviews is "  + str(nonvinereviews))
print("The total count of 5 star Non-Vine watch reviews is "  + str(fivestarreviewsnonvine))
print(f"The percent of 5 star Vine watch reviews is "  + "{0:.1%}".format(percentvinefive))
print(f"The percent of 5 star Non-Vine watch reviews is "  + "{0:.1%}".format(percentnonvinefive))

The total count of watch reviews is 8409
The total count of 5 star watch reviews is 4347
The total count of Vine watch reviews is 47
The total count of 5 star Vine watch reviews is 15
The total count of Non-Vine watch reviews is 8362
The total count of 5 star Non-Vine watch reviews is 4332
The percent of 5 star Vine watch reviews is 31.9%
The percent of 5 star Non-Vine watch reviews is 51.8%


The percent of 5 star Vine watch reviews is 31.9%
The percent of 5 star Non-Vine watch reviews is 51.8%
